In [33]:
import pandas as pd
import json

articles = pd.read_csv("articles_with_cluster.tsv", sep = "\t")

In [34]:
multiple_maxima = []
clear_articles = []
count_ones = 0 
only_one_kw = 0

topics = [topic for topic in articles.columns if topic not in ["id", "title"]]
for article in articles.to_dict("records"):
    maximum = max([score for topic, score in article.items() if topic in topics])
    if maximum == 1:
        count_ones +=1
    if sum([score for topic, score in article.items() if topic in topics]) == 1:
        only_one_kw +=1
    if list(article.values()).count(maximum)>1:
        multiple_maxima.append(article)
    else:
        clear_articles.append(article)


final_articles = clear_articles + pd.read_csv("classified_multiple_maxima_topics.tsv", sep ="\t").to_dict("records")

In [35]:
with open("corpus.json", "r") as f:
    corpus = json.load(f)

def get_topic(article):
    maximum = max([score for topic, score in article.items() if topic in topics])
 
    for topic, score in article.items():
        if score == maximum:
            return topic
            

for article in final_articles:
    article["topic"] = get_topic(article)
    for art in corpus:
        if article["id"] == art["id"]:
            article["date"] = art["date"]


In [23]:
pd.DataFrame(multiple_maxima).to_csv("multiple_maxima_topics.tsv", sep="\t", index=False)